# Confluent notebook
> Notebook to store confluent related functionalities

In [ ]:
# | default_exp confluent

In [ ]:
# | export

from os import environ
from pathlib import Path
from typing import *

from aiokafka.helpers import create_ssl_context
from airt.logger import get_logger
from confluent_kafka.admin import AdminClient, NewTopic

In [ ]:
from time import sleep

from airt_service.db.models import create_user_for_testing

In [ ]:
test_username = create_user_for_testing()
display(test_username)

'mrvjqtxdzr'

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
# | export

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_bootstrap_servers = f":{kafka_server_port},".join(kafka_server_url.split(",")) + f":{kafka_server_port}"

aio_kafka_config = {
    "bootstrap_servers": kafka_bootstrap_servers,
    "group_id": f"{kafka_bootstrap_servers}_as_group",
    "auto_offset_reset": "earliest",
}
if "KAFKA_API_KEY" in environ:
    aio_kafka_config = {
        **aio_kafka_config,
        **{
            "security_protocol": "SASL_SSL",
            "sasl_mechanism": environ["KAFKA_SASL_MECHANISM"],
            "sasl_plain_username": environ["KAFKA_API_KEY"],
            "sasl_plain_password": environ["KAFKA_API_SECRET"],
            "ssl_context": create_ssl_context(),
        },
    }

In [ ]:
for key, _ in aio_kafka_config.items():
    assert "_" in key and "." not in key, key
aio_kafka_config

{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092',
 'group_id': 'kumaran-airt-service-kafka-1:9092_group',
 'auto_offset_reset': 'earliest'}

In [ ]:
# | export

# confluent_kafka_config = {key.replace("_", "."):value for key, value in aio_kafka_config.items()}
confluent_kafka_config = {
    key.replace("_", "."): aio_kafka_config[key]
    for key in ["bootstrap_servers", "group_id", "auto_offset_reset"]
}
if "KAFKA_API_KEY" in environ:
    confluent_kafka_config["security.protocol"] = aio_kafka_config["security_protocol"]
    confluent_kafka_config["sasl.mechanisms"] = aio_kafka_config["sasl_mechanism"]
    confluent_kafka_config["sasl.username"] = aio_kafka_config["sasl_plain_username"]
    confluent_kafka_config["sasl.password"] = aio_kafka_config["sasl_plain_password"]

In [ ]:
for key, _ in confluent_kafka_config.items():
    assert "_" not in key and "." in key, key
confluent_kafka_config

{'bootstrap.servers': 'kumaran-airt-service-kafka-1:9092',
 'group.id': 'kumaran-airt-service-kafka-1:9092_group',
 'auto.offset.reset': 'earliest'}

In [ ]:
# | export


def get_topic_names_to_create(username: str) -> List[str]:
    """
    Get a list of topic names to create for given username

    Args:
        username: username of user for whom the list of topic names is required
    Returns:
        A list of topic names unique to the username
    """
    topic_names = [
        "start_training_data",
        "training_data",
        "realtime_data",
        "training_data_status",
        "training_model_status",
        "model_metrics",
        "prediction",
    ]
    return [f"{username}_{topic_name}" for topic_name in topic_names]

In [ ]:
expected = [
    f"{test_username}_start_training_data",
    f"{test_username}_training_data",
    f"{test_username}_realtime_data",
    f"{test_username}_training_data_status",
    f"{test_username}_training_model_status",
    f"{test_username}_model_metrics",
    f"{test_username}_prediction",
]
actual = get_topic_names_to_create(username=test_username)
assert actual == expected, actual
actual

['mrvjqtxdzr_start_training_data',
 'mrvjqtxdzr_training_data',
 'mrvjqtxdzr_realitime_data',
 'mrvjqtxdzr_training_data_status',
 'mrvjqtxdzr_training_model_status',
 'mrvjqtxdzr_model_metrics',
 'mrvjqtxdzr_prediction']

In [ ]:
# | export


def create_topics_for_user(username: str) -> None:
    """
    Create necessary topics for given user

    Args:
        username: username of user for whom the topics needs to be created
    """

    topic_names_to_create = get_topic_names_to_create(username)
    admin_client = AdminClient(confluent_kafka_config)

    num_partitions = 6
    replication_factor = 3 if "KAFKA_API_KEY" in environ else 1

    existing_topics = admin_client.list_topics().topics

    topics_to_create = [
        NewTopic(topic_name, num_partitions, replication_factor)
        for topic_name in topic_names_to_create
        if topic_name not in existing_topics
    ]
    if not topics_to_create:
        return

    futures = admin_client.create_topics(topics_to_create)

    for topic, future in futures.items():
        try:
            future.result()
            logger.info(f"Topic {topic} created")
        except Exception as e:
            logger.error(f"Topic {topic} creation failed")
            raise e

In [ ]:
topic_names = get_topic_names_to_create(username=test_username)
create_topics_for_user(username=test_username)

sleep(1)

admin_client = AdminClient(confluent_kafka_config)
topic_metadata = admin_client.list_topics()

for topic_name in topic_names:
    assert (
        topic_metadata.topics.get(topic_name) is not None
    ), f"Topic {topic_name} not found"

23-01-12 06:29:39.570 [INFO] __main__: Topic mrvjqtxdzr_start_training_data created
23-01-12 06:29:39.571 [INFO] __main__: Topic mrvjqtxdzr_training_data created
23-01-12 06:29:39.571 [INFO] __main__: Topic mrvjqtxdzr_realitime_data created
23-01-12 06:29:39.572 [INFO] __main__: Topic mrvjqtxdzr_training_data_status created
23-01-12 06:29:39.573 [INFO] __main__: Topic mrvjqtxdzr_training_model_status created
23-01-12 06:29:39.574 [INFO] __main__: Topic mrvjqtxdzr_model_metrics created
23-01-12 06:29:39.575 [INFO] __main__: Topic mrvjqtxdzr_prediction created


%4|1673504979.449|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1673504979.449|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1673504980.580|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1673504980.580|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


In [ ]:
# | export


def delete_topics_for_user(username: str) -> None:
    """
    Delete necessary topics for given user

    Args:
        username: username of user for whom the topics needs to be deleted
    """

    topic_names_to_delete = get_topic_names_to_create(username)
    admin_client = AdminClient(confluent_kafka_config)

    existing_topics = admin_client.list_topics().topics

    topics_to_delete = [
        topic_name
        for topic_name in topic_names_to_delete
        if topic_name in existing_topics
    ]
    if not topics_to_delete:
        return

    futures = admin_client.delete_topics(topics_to_delete)

    for topic, future in futures.items():
        try:
            future.result()
            logger.info(f"Topic {topic} deleted")
        except Exception as e:
            logger.error(f"Topic {topic} deletion failed")
            raise e

In [ ]:
topic_names = get_topic_names_to_create(username=test_username)
delete_topics_for_user(username=test_username)

sleep(1)

admin_client = AdminClient(confluent_kafka_config)
topic_metadata = admin_client.list_topics()

for topic_name in topic_names:
    assert (
        topic_metadata.topics.get(topic_name) is None
    ), f"Topic {topic_name} found and not deleted"

23-01-12 06:29:40.617 [INFO] __main__: Topic mrvjqtxdzr_start_training_data deleted
23-01-12 06:29:40.617 [INFO] __main__: Topic mrvjqtxdzr_training_data deleted
23-01-12 06:29:40.618 [INFO] __main__: Topic mrvjqtxdzr_realitime_data deleted
23-01-12 06:29:40.619 [INFO] __main__: Topic mrvjqtxdzr_training_data_status deleted
23-01-12 06:29:40.619 [INFO] __main__: Topic mrvjqtxdzr_training_model_status deleted
23-01-12 06:29:40.620 [INFO] __main__: Topic mrvjqtxdzr_model_metrics deleted
23-01-12 06:29:40.620 [INFO] __main__: Topic mrvjqtxdzr_prediction deleted


%4|1673504980.595|CONFWARN|rdkafka#producer-3| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1673504980.595|CONFWARN|rdkafka#producer-3| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1673504981.624|CONFWARN|rdkafka#producer-4| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1673504981.624|CONFWARN|rdkafka#producer-4| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
